In [97]:

#Python program to scrape website  
#and save quotes from website 
import requests 
from bs4 import BeautifulSoup
import csv
import json

   
URL = "http://www.talendforge.org/components/index.php?version=255&edition=63&showAll=1"

r = requests.get(URL) 
   
#soup = BeautifulSoup(r.content, 'html5lib') 
soup = BeautifulSoup(r.content, 'html.parser') 
#print(soup)

In [102]:
components=[]  # a list to store quotes 
headers = ['Name', 'Image', 'Description', 'Help']
import pandas as pd
df = pd.DataFrame(columns=headers)
df
#table = soup.find_all('tr', attrs = {'class':'table-info'})  
table = soup.find('table')
#print(table).prettify()
rows = table.find('tbody').find_all('tr') 
record_counter = 0
for row in rows:
    #print('###', row)
    elements = row.find_all('td')
    length = len(elements)
    i = 0
    for e in elements:
        if length == 1:
            category = e.text.strip()
            #print('Category = ', e.text)
        if length > 4:
            i += 1
            match i:
                case 1:
                    image = e.img['src']
                case 2:
                    help = e.a['href']
                    name = e.text.strip()
                case 3:
                    description = e.text.strip()
                case 4:
                    #print('Adding category', category)
                    record_counter += 1
                    components.append([name,category,image,description,help])
                case _:
                    break

print('# of components = ', record_counter)


# of components =  1286


In [101]:

# Output:
# This will create a file named 'components.json' and write the JSON data into it.
with open('components.json', 'w') as f:
    json.dump(components, f)

f.close()


In [134]:
import shutil
import os
#
# get icon file and write it locally; this takes awhile so we only do it once
# but we only do this once as it is slow 8)
#
# need to create the images directory first
#
def get_image(base_string):
    filename = os.path.basename(base_string)
    
    url = "https://talendforge.org" + base_string
    print('Fetching -> ', url)
    response = requests.get(url, stream=True)
    with open('images/'+filename, 'wb') as f2:
        shutil.copyfileobj(response.raw, f2)
        f2.close()
    del response

# read json file

# Opening JSON file
f = open('components.json')

# returns JSON object as a dictionary
data = json.load(f)

# Iterating through the json list
name = 0
category = 1
image = 2
description = 3
help = 4


for element in data:
    get_image(element[image])
#    print('Fetching icon image - ', element[image])

#get_image('https://talendforge.org/images/8.0.1/cAWSConnection_icon32.png')
#
# Closing file
#
f.close()



Fetching ->  https://talendforge.org/images/8.0.1/cAWSConnection_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/cAWSS3_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/cAWSSES_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/cAWSSNS_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/cAWSSQS_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/tBigQueryBulkExec_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/tBigQueryInput_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/tBigQueryOutput_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/tBigQueryOutputBulk_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/tBigQuerySQLRow_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/tCassandraBulkExec_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/tCassandraClose_icon32.png
Fetching ->  https://talendforge.org/images/8.0.1/tCassandraConnection_icon32.png
Fetching ->  https:/

In [133]:
#
# do one for a test to see if it works
#
import shutil

import requests

url = 'https://talendforge.org/images/8.0.1/cAWSConnection_icon32.png'
response = requests.get(url, stream=True)
with open('cAWSConnection_icon32.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
del response



In [20]:
import json
from pprint import pprint
# read json file

# open JSON file
f = open('components.json')

# returns JSON object as a dictionary
data = json.load(f)

# Iterating through the json list
name = 0
category = 1
image = 2
description = 3
help = 4
children = []
level1Dict = {}
level2Dict = {}

def printList(*args):
    for a in args:
        print(a)
    return 
def buildBaseDict(element):
    component = {}
    component['id']    = element[name]
    component['image'] = element[image]
    component['description']  = element[description]
    component['help']  = element[help]
    #pprint(children)
    # return dictionary
    return component
    
currentCategory = None
component = {}

for element in data:
    if currentCategory == None:
        currentCategory = element[category]
    if currentCategory != element[category]:
        level2Dict['children'] = children
        level1Dict['data'] = currentCategory
        currentCategory = element[category]
        children = []
        
    componentDict = buildBaseDict(element)
    # append component dictionary to the children list
    children.append(componentDict)
    
pprint(level1Dict)

#printList(component)
#printList(element)
#    print('Fetching icon image - ', element[image])

#
# Closing file
#
f.close()


{'data': 'Webservice'}


In [20]:
#
# Attribution: https://www.lavivienpost.com/build-hierarchy-tree/
# Thank you!

class Component:
    children = []

    # Constructor
    def __init__(self, id, name, parentId, image, description, help):
        self.id = id
        self.name = name
        self.parentId = parentId
        if image :
            self.image = image
            self.description = description
            self.help = help
            
class BuildHierarchyTree:
    components = {}
    root = None

    def printList(*args):
        for a in args:
            print(a)
        return 
    
    # Read data and build map
    def readDataAndCreateMap(self, data):
        component = None
        for d in data:
            values = d.split("|") # use | as delimiter
            vLength = len(values)
            if vLength == 3:
                component = Component(values[0], values[1], values[2])
            elif vLength == 4:
                component = Component(values[0], values[1], values[2], values[3])
            else:
                component = Component(values[0], values[1], 0)
            self.components[component.id] = component
            if component.parentId == 0 :
                self.root = component
          
    # Build tree
    def buildHierarchyTree(self, root):
        component = root
        children = self.getChildrenById(component.id)
        component.children = children
        if len(children) == 0 :
            return
        for child in children:
            self.buildHierarchyTree(child)
            
    # Get children list by given id
    def getChildrenById(self, parentId):
        children = []
        for c in self.components.values():
            if c.parentId == parentId:
                children.append(c)
        return children

            
    # Print tree
    def printTree(self, root, level) :
        for i in range(0, level, 1) :
            print("\t", end='')
        print(root.id, root.name);
        children = root.children
        for c in children:
            self.printTree(c, level+1)

data = {
    "1|Rob 'the mob' Choi|6",
    "2|Paul Blue|5",
    "3|Lois Lemer|6",
    "4|Chris Jay|5",
    "5|Moira Medi|6",
    "6|Paul R"
}
testdata = {
    "1|Talend Studio 8.0.1",
    "2|AWS|1",
    "3|AWSConnection|2|https://talendforge.org/images/8.0.1/generic_component_icon32.png",
    "4|AWSS3|2|https://talendforge.org/images/8.0.1/generic_component_icon32.png",
    "5|AWSS3Bucket|2|https://talendforge.org/images/8.0.1/generic_component_icon32.png",
    "6|Business Intelligence|1"
}
testdata2 = {
    "1|Talend Studio 8.0.1||||",
    "2|AWS|1|||",
    "3|tBigQueryOutput|2|'/images/8.0.1/tBigQueryOutput_icon32.png'|'Transfers the data provided by its preceding component to Google BigQuery.'|'https://help.talend.com/access/sources/content/topic?pageid=tbigqueryoutput&EnrichVersion=8.0&afs:lang=en'"
}

tree = BuildHierarchyTree()
tree.readDataAndCreateMap(testdata2)
tree.buildHierarchyTree(tree.root)
#tree.printTree(tree.root, 0)
    
        

TypeError: Component.__init__() missing 3 required positional arguments: 'image', 'description', and 'help'